In [1]:
import random

import numpy as np
import pandas as pd
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adamax, RMSprop, SGD
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import StandardScaler, RobustScaler
from itertools import combinations
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from keras.utils import plot_model
from keras import metrics
import talos as ta
from talos.model.normalizers import lr_normalizer
import statsmodels.api as sm

Using TensorFlow backend.


In [6]:
rw = pd.read_csv("data/winequality-red.csv", sep=";")
ww = pd.read_csv("data/winequality-white.csv", sep=";")
ww['type'] = '0'
rw['type'] = '1'
dataset = "ww"
#data = rw.append(ww)
data = (ww, rw)[dataset=='rw']

In [56]:



def pick_features(data, num_features, one_hot=True, binary=False):
    if one_hot:
        label = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        # label = [3, 4, 5, 6, 7, 8]
        dummies = data['quality'].astype(CategoricalDtype(categories=label))
        dummies = pd.get_dummies(dummies, prefix='quality', prefix_sep='_')
        data = pd.concat([data, dummies], axis=1)
        label_names = []
        for i in label:
            label_names.append('quality_' + str(i))
    else:
        if (binary):
            label_names = ['type']
        else:
            label_names = ['quality']

    comb = []
    for i in list(combinations(range(1, 12), num_features)):
        comb.append(i)

    return data, label_names, comb


def top_2_categorical_accuracy(ytrue, ypred):
    return metrics.top_k_categorical_accuracy(ytrue, ypred, k=2)

def nn_softmax(data, num_features, ss=False):
    column_names_lookup = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides',
                           'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']

    data, label_names, comb  = pick_features(data, num_features, one_hot=True)

    for columns in comb:
        column_names = []
        for i in columns:
            column_names.append(column_names_lookup[i-1])

        if ss:
            print("Using StandardScaler")
            X = StandardScaler().fit_transform(data[column_names])
            # X = RobustScaler().fit_transform(np.array(data[column_names]))
        else:
            X = data[column_names]

        y = np.array(data[label_names])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

        p = {'activation': ['relu', 'sigmoid'],
             'optimizer': ['Adamax', 'Adam'],
             'losses': ['categorical_crossentropy'],
             'batch_size': [32, 64, 128],
             'epochs': [200, 300, 400],
             'lr': [0.001]}
        
        scores = []
        def yawda_model(X_train, y_train, x_val, y_val, params):
            act = params['activation']
            model = Sequential()
            model.add(Dense(128, activation=act, input_dim=X_train.shape[1], kernel_initializer='he_normal'))
            model.add(Dropout(0.5))
            model.add(Dense(128, activation=act))
            model.add(Dropout(0.5))
            model.add(Dense(128, activation=act))
            model.add(Dropout(0.5))
            model.add(Dense(128, activation=act))
            model.add(Dropout(0.5))
            model.add(Dense(y_train.shape[1], activation='softmax'))
            model.compile(loss=params['losses'], optimizer=params['optimizer'], metrics=[top_2_categorical_accuracy])

            history = model.fit(np.array(X_train), np.array(y_train), epochs=params['epochs'], batch_size=params['batch_size'], verbose=0, validation_data=[x_val, y_val])
            scores.append(model.evaluate(np.array(X_test), np.array(y_test), batch_size=1))
            
            return history, model

        scan_object = ta.Scan(X_train, y_train, model=yawda_model, params=p, experiment_name='yawda', val_split=0.3)
        return scan_object, scores
        


In [53]:
scan_object, scores=nn_softmax(rw, 11, ss=True)
scan_object.data







  0%|                                                                                           | 0/36 [00:00<?, ?it/s]

Using StandardScaler
320/320 [==============================] - 1s 2ms/step








  3%|██▎                                                                                | 1/36 [00:18<11:02, 18.93s/it]

320/320 [==============================] - 0s 1ms/step








  6%|████▌                                                                              | 2/36 [00:36<10:25, 18.39s/it]

...

320/320 [==============================] - 1s 2ms/step








 97%|███████████████████████████████████████████████████████████████████████████████▋  | 35/36 [10:16<00:11, 11.96s/it]

320/320 [==============================] - 0s 1ms/step








100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [10:29<00:00, 17.50s/it]


round_epochs      loss  top_2_categorical_accuracy activation  batch_size  \
0            200  0.905602                    0.901676       relu          32   
1            200  0.805863                    0.927374       relu          32   
2            300  0.869875                    0.902793       relu          32   
3            300  0.718401                    0.945251       relu          32   
4            400  0.803199                    0.925140       relu          32   
5            400  0.649329                    0.953073       relu          32   
6            200  0.901070                    0.901676       relu          64   
7            200  0.879810                    0.901676       relu          64   
8            300  0.866134                    0.910615       relu          64   
9            300  0.758828                    0.931844       relu          64   
10           400  0.832391                    0.920670       relu          64   
11           400  0.711204                    0.936313       relu          64   
12           200  0.969133                    0.888268       relu         128   
13           200  0.885573                    0.899441       relu         128   
14           300  0.901073                    0.903911       relu         128   
15           300  0.829103                    0.909497       relu         128   
16           400  0.900463                    0.898324       relu         128   
17           400  0.760450                    0.941899       relu         128   
18           200  0.939733                    0.897207    sigmoid          32   
19           200  0.897303                    0.903911    sigmoid          32   
20           300  0.916147                    0.901676    sigmoid          32   
21           300  0.861920                    0.915084    sigmoid          32   
22           400  0.899823                    0.899441    sigmoid          32   
23           400  0.850497                    0.917318    sigmoid          32   
24           200  0.958976                    0.896089    sigmoid          64   
25           200  0.922611                    0.898324    sigmoid          64   
26           300  0.932824                    0.903911    sigmoid          64   
27           300  0.896527                    0.906145    sigmoid          64   
28           400  0.894613                    0.893855    sigmoid          64   
29           400  0.868502                    0.917318    sigmoid          64   
30           200  0.997015                    0.891620    sigmoid         128   
31           200  0.952459                    0.899441    sigmoid         128   
32           300  0.974071                    0.894972    sigmoid         128   
33           300  0.926119                    0.897207    sigmoid         128   
34           400  0.950743                    0.898324    sigmoid         128   
35           400  0.891409                    0.909497    sigmoid         128   

    epochs                    losses     lr optimizer  
0      200  categorical_crossentropy  0.001    Adamax  
1      200  categorical_crossentropy  0.001      Adam  
2      300  categorical_crossentropy  0.001    Adamax  
3      300  categorical_crossentropy  0.001      Adam  
4      400  categorical_crossentropy  0.001    Adamax  
5      400  categorical_crossentropy  0.001      Adam  
6      200  categorical_crossentropy  0.001    Adamax  
7      200  categorical_crossentropy  0.001      Adam  
8      300  categorical_crossentropy  0.001    Adamax  
9      300  categorical_crossentropy  0.001      Adam  
10     400  categorical_crossentropy  0.001    Adamax  
11     400  categorical_crossentropy  0.001      Adam  
12     200  categorical_crossentropy  0.001    Adamax  
13     200  categorical_crossentropy  0.001      Adam  
14     300  categorical_crossentropy  0.001    Adamax  
15     300  categorical_crossentropy  0.001      Adam  
16     400  categorical_crossentropy  0.001    Adamax 

In [55]:
scores

[[0.9720296889077872, 0.9],
 [0.9941061101271771, 0.884375],
 [0.9578112975461408, 0.896875],
 [0.9951075598422904, 0.875],
 [0.970788705162704, 0.9],
 [1.0491287023731275, 0.875],
 [0.9824503318872303, 0.915625],
 [0.9650404840474949, 0.909375],
 [0.9729361115954817, 0.9],
 [1.009417297516484, 0.8875],
 [0.9776401529554277, 0.903125],
 [1.041001563220925, 0.8875],
 [0.9810010823886841, 0.9],
 [0.9755614651134238, 0.9],
 [0.9749324049567804, 0.9125],
 [0.9661998018156737, 0.89375],
 [0.9698137240950018, 0.903125],
 [0.9929385778610594, 0.884375],
 [0.9405425488948822, 0.909375],
 [0.933200971595943, 0.915625],
 [0.9381815506145358, 0.9125],
 [0.9489266918972135, 0.9125],
 [0.9365087080746889, 0.9125],
 [0.9428765517193824, 0.9125],
 [0.9439799438696355, 0.903125],
 [0.9381510528270155, 0.90625],
 [0.9448609053622932, 0.909375],
 [0.9386815877165645, 0.909375],
 [0.9392706351354718, 0.915625],
 [0.9454154551029206, 0.909375],
 [0.9430316201411187, 0.909375],
 [0.942610120261088, 0.9125]

In [58]:
analyze_object=ta.Analyze(scan_object)
analyze_object.best_params('val_top_2_categorical_accuracy', ['top_2_categorical_accuracy', 'loss', 'val_loss'])


array([['relu', 0.001, 400, 32, 'categorical_crossentropy', 'Adam', 400,
        0],
       ['relu', 0.001, 300, 32, 'categorical_crossentropy', 'Adam', 300,
        1],
       ['relu', 0.001, 400, 128, 'categorical_crossentropy', 'Adam', 400,
        2],
       ['relu', 0.001, 400, 64, 'categorical_crossentropy', 'Adam', 400,
        3],
       ['relu', 0.001, 300, 64, 'categorical_crossentropy', 'Adam', 300,
        4],
       ['relu', 0.001, 200, 32, 'categorical_crossentropy', 'Adam', 200,
        5],
       ['relu', 0.001, 400, 32, 'categorical_crossentropy', 'Adamax',
        400, 6],
       ['relu', 0.001, 400, 64, 'categorical_crossentropy', 'Adamax',
        400, 7],
       ['sigmoid', 0.001, 400, 64, 'categorical_crossentropy', 'Adam',
        400, 8],
       ['sigmoid', 0.001, 400, 32, 'categorical_crossentropy', 'Adam',
        400, 9]], dtype=object)

In [57]:
scan_object2, scores2 =nn_softmax(ww, 11, ss=True)
scan_object2.data







  0%|                                                                                           | 0/36 [00:00<?, ?it/s]

Using StandardScaler
980/980 [==============================] - 1s 925us/step








  3%|██▎                                                                                | 1/36 [00:55<32:07, 55.06s/it]

980/980 [==============================] - 1s 800us/step








  6%|████▌                                                                              | 2/36 [02:00<32:59, 58.21s/it]

...

980/980 [==============================] - 1s 708us/step








 97%|███████████████████████████████████████████████████████████████████████████████▋  | 35/36 [41:17<00:43, 43.22s/it]

980/980 [==============================] - 1s 968us/step








100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [42:04<00:00, 70.11s/it]


round_epochs  val_loss  val_top_2_categorical_accuracy      loss  \
0            200  1.030079                        0.880952  1.003130   
1            200  1.019710                        0.880952  0.948352   
2            300  1.027988                        0.882653  0.976537   
3            300  1.012669                        0.881803  0.912669   
4            400  1.012959                        0.879252  0.940246   
5            400  1.011771                        0.882653  0.876624   
6            200  1.047262                        0.878401  1.024925   
7            200  1.020708                        0.880952  0.969990   
8            300  1.024254                        0.879252  0.992195   
9            300  1.015637                        0.880102  0.909005   
10           400  1.021475                        0.884354  0.958221   
11           400  1.030890                        0.884354  0.867203   
12           200  1.046587                        0.879252  1.046238   
13           200  1.020335                        0.880102  0.992163   
14           300  1.036524                        0.878401  1.007921   
15           300  1.016651                        0.883503  0.937818   
16           400  1.035500                        0.877551  0.985936   
17           400  1.014323                        0.879252  0.907978   
18           200  1.045057                        0.870748  1.052080   
19           200  1.051497                        0.877551  1.019058   
20           300  1.041947                        0.880102  1.038645   
21           300  1.050508                        0.882653  0.987275   
22           400  1.039215                        0.882653  1.012320   
23           400  1.054592                        0.869898  0.968207   
24           200  1.049836                        0.865646  1.078356   
25           200  1.048761                        0.878401  1.033691   
26           300  1.040853                        0.881803  1.038266   
27           300  1.047341                        0.874150  1.008854   
28           400  1.046315                        0.878401  1.027047   
29           400  1.051358                        0.882653  0.991828   
30           200  1.068241                        0.862245  1.079231   
31           200  1.047831                        0.869898  1.050400   
32           300  1.047321                        0.869048  1.065970   
33           300  1.043569                        0.876701  1.020996   
34           400  1.038127                        0.878401  1.038856   
35           400  1.047708                        0.879252  1.010465   

    top_2_categorical_accuracy activation  batch_size  epochs  \
0                     0.870168       relu          32     200   
1                     0.882567       relu          32     200   
2                     0.881109       relu          32     300   
3                     0.882567       relu          32     300   
4                     0.883662       relu          32     400   
5                     0.890956       relu          32     400   
6                     0.865427       relu          64     200   
7                     0.881838       relu          64     200   
8                     0.870897       relu          64     300   
9                     0.896791       relu          64     300   
10                    0.882932       relu          64     400   
11                    0.895697       relu          64     400   
12                    0.861415       relu         128     200   
13                    0.870532       relu         128     200   
14                    0.870532       relu         128     300   
15                    0.883662       relu         128     300   
16                    0.871991       relu         128     400   
17                    0.892779       relu         128     400   
18                    0.855580    sigmoid          32     200   
19                    0.863603    sigmo

In [59]:
scores2

[[1.093133395682184, 0.8510204081632653],
 [1.0666362035183274, 0.8551020408163266],
 [1.0746882570945486, 0.8530612244897959],
 [1.0616252525804601, 0.860204081632653],
 [1.0709869425135607, 0.8540816326530613],
 [1.0530953774380745, 0.8653061224489796],
 [1.0954354917212408, 0.8530612244897959],
 [1.065030192880302, 0.860204081632653],
 [1.0801105751979108, 0.8510204081632653],
 [1.071024656592279, 0.8571428571428571],
 [1.0709380275284757, 0.8581632653061224],
 [1.056468134534032, 0.8561224489795919],
 [1.1044638906510509, 0.8448979591836735],
 [1.0741751225931304, 0.8520408163265306],
 [1.0861722149411026, 0.8448979591836735],
 [1.07542345254701, 0.8693877551020408],
 [1.081531326145846, 0.8571428571428571],
 [1.0568696396007222, 0.8642857142857143],
 [1.1127097978275649, 0.8377551020408164],
 [1.124852989401136, 0.8448979591836735],
 [1.1077584409896208, 0.8479591836734693],
 [1.113720997377318, 0.8469387755102041],
 [1.1084313430652326, 0.8459183673469388],
 [1.1372820939488557, 

In [60]:
analyze_object2=ta.Analyze(scan_object2)
analyze_object2.best_params('val_top_2_categorical_accuracy', ['top_2_categorical_accuracy', 'loss', 'val_loss'])


array([['relu', 0.001, 400, 64, 'categorical_crossentropy', 'Adamax',
        400, 0],
       ['relu', 0.001, 400, 64, 'categorical_crossentropy', 'Adam', 400,
        1],
       ['relu', 0.001, 300, 128, 'categorical_crossentropy', 'Adam', 300,
        2],
       ['relu', 0.001, 300, 32, 'categorical_crossentropy', 'Adamax',
        300, 3],
       ['sigmoid', 0.001, 400, 64, 'categorical_crossentropy', 'Adam',
        400, 4],
       ['relu', 0.001, 400, 32, 'categorical_crossentropy', 'Adam', 400,
        5],
       ['sigmoid', 0.001, 400, 32, 'categorical_crossentropy', 'Adamax',
        400, 6],
       ['sigmoid', 0.001, 300, 32, 'categorical_crossentropy', 'Adam',
        300, 7],
       ['relu', 0.001, 300, 32, 'categorical_crossentropy', 'Adam', 300,
        8],
       ['sigmoid', 0.001, 300, 64, 'categorical_crossentropy', 'Adamax',
        300, 9]], dtype=object)

In [20]:
evaluate_object = ta.Evaluate(scan_object2)
evaluate_object.evaluate(X_test, y_test, folds=16, metric='top_2_categorical_accuracy', task='multi_label')

[0.3235156615240767,
 0.331063321385902,
 0.4868370112272551,
 0.4282512733446519,
 0.4678177960686076,
 0.5003847678567264,
 0.2702380952380952,
 0.39944444444444444,
 0.42456429193028666,
 0.3568262411347518,
 0.2730329311211664,
 0.3569444444444444,
 0.2932472423997848,
 0.4459383753501401,
 0.3074975574529544,
 0.3202997719126751]